## Data Preprocessing Scripts

### Process the Original birds data with exploded dates

In [2]:
import pandas as pd
def preprocess_bird_data(file_path, sheet_name):

    df = pd.read_excel(file_path, sheet_name=sheet_name)
    print(df.columns)  # Check all column names

    # Create two new columns 'From_Date' and 'To_Date' to accommodate the date ranges
    date_ranges = df['Date'].apply(lambda x: pd.Series(parse_date_ranges(str(x)), index=['From_Date', 'To_Date']))
    df = pd.concat([df.drop('Date', axis=1), date_ranges], axis=1)

    # Explode the DataFrame by creating a new row for each date within the date range
    exploded_dates = df.apply(lambda row: explode_date_ranges(row['From_Date'], row['To_Date']), axis=1)
    exploded_dates.name = 'Exploded_Dates'
    df = df.join(exploded_dates)

    # Drop the 'From_Date' and 'To_Date' columns as they are no longer needed
    df.drop(['From_Date', 'To_Date'], axis=1, inplace=True)

    # Explode the DataFrame to have a row for each date in the range
    df = df.explode('Exploded_Dates')
    df.rename(columns={'Exploded_Dates': 'Observation_Date'}, inplace=True)

    # Convert 'Observation_Date' to datetime and extract year, month, and day
    df['Observation_Date'] = pd.to_datetime(df['Observation_Date'], utc=False)
    df['Year'] = df['Observation_Date'].dt.year
    df['Month'] = df['Observation_Date'].dt.month
    df['Day'] = df['Observation_Date'].dt.day

    # Save the preprocessed data to a new Excel file
    processed_file_path = '/home/ai-user/NUS_IS/project/processed_bird_observations.xlsx' # Replace with the path to your Excel file
    df.to_excel(processed_file_path, index=False)

    return processed_file_path

# Define the function to handle date ranges
def parse_date_ranges(date_entry):
    date_range = date_entry.split(' to ')
    start_date = pd.to_datetime(date_range[0], dayfirst=True, errors='coerce')
    end_date = pd.to_datetime(date_range[-1], dayfirst=True, errors='coerce') if len(date_range) > 1 else start_date
    return start_date, end_date

# Define the function to explode date ranges into a list of dates
def explode_date_ranges(start_date, end_date):
    if pd.isna(start_date) or pd.isna(end_date):
        return []
    return pd.date_range(start_date, end_date).tolist()

# You can call this function with the path to your Excel file and the sheet name
file_path = '/home/ai-user/NUS_IS/project/Birds_data.xlsx'  # Replace with the path to your Excel file
sheet_name = 'Sheet1'  # Replace with your actual sheet name
output_path = preprocess_bird_data(file_path, sheet_name)
print(f"Processed data saved to: {output_path}")


### For Filter from the Data Range

In [ ]:
import pandas as pd

# Load the dataset
df = pd.read_excel('processed_bird_observations.xlsx')

# Ensure Observation_Date is in datetime format
df['Observation_Date'] = pd.to_datetime(df['Observation_Date'])

# Define the start and end date for the filter
start_date = pd.Timestamp('2018-10-05')
end_date = pd.Timestamp('2024-02-29')

# Filter the DataFrame for the given date range
filtered_df = df[(df['Observation_Date'] >= start_date) & (df['Observation_Date'] <= end_date)]

# Save the filtered data to a new Excel file
filtered_df.to_excel('filtered_data.xlsx', index=False)

print("Filtered data saved to filtered_data.xlsx")

### For including "NONE" species on Non-predicted dates

In [ ]:
import pandas as pd

# Load the dataset
df = pd.read_excel('filtered_data.xlsx')

# Ensure Observation_Date is in datetime format
df['Observation_Date'] = pd.to_datetime(df['Observation_Date'])

# Generate a complete date range from October 5, 2018, to January 29, 2024
full_date_range = pd.date_range(start='2018-10-05', end='2024-01-29')

# Identify missing dates from the original dataset
missing_dates = full_date_range.difference(df['Observation_Date'])
print(missing_dates)

# Create additional rows for missing dates
missing_rows = pd.DataFrame({
    'Species': 'No_Species_Found',
    'Location': 'None',
    'Observation_Date': missing_dates,
    'Year': missing_dates.year,
    'Month': missing_dates.month,
    'Day': missing_dates.day
})

# Combine the original dataset with the additional rows
combined_df = pd.concat([df, missing_rows], ignore_index=True)

# Sort the combined dataset by Observation_Date
combined_df.sort_values(by='Observation_Date', inplace=True)

# Add additional columns for Year, Month, and Day based on Observation_Date
combined_df['Year'] = combined_df['Observation_Date'].dt.year
combined_df['Month'] = combined_df['Observation_Date'].dt.month
combined_df['Day'] = combined_df['Observation_Date'].dt.day

# Save the sorted and augmented dataset to a new Excel file
combined_df.to_excel('augmented_data.xlsx', index=False)

print("Augmented data saved to augmented_data.xlsx")

### For combining Excel data

In [ ]:
import pandas as pd

# Load datasets
augmented_df = pd.read_excel('augmented_data.xlsx')
combined_df = pd.read_excel('overall_data.xlsx')

# Ensure datetime compatibility
augmented_df['Observation_Date'] = pd.to_datetime(augmented_df['Observation_Date'])
combined_df['datetime'] = pd.to_datetime(combined_df['datetime'])

# List to collect all new rows
new_rows = []

# Iterate through unique dates in augmented data
for date in augmented_df['Observation_Date'].unique():
    # Find matching row in combined data
    matching_row = combined_df[combined_df['datetime'] == date]
    
    # Check for matching rows in combined data
    if not matching_row.empty:
        # Extract matching row data as a dictionary
        matching_data = matching_row.iloc[0].to_dict()
        
        # Find all entries for this date in augmented data
        date_entries = augmented_df[augmented_df['Observation_Date'] == date]
        
        # Replicate combined data for each entry in augmented data
        for _, row in date_entries.iterrows():
            # Combine data from both rows
            new_row = {**row.to_dict(), **matching_data}
            # Adjust the 'datetime' field to match 'Observation_Date'
            new_row['datetime'] = new_row['Observation_Date']
            # Add the new combined row to the list
            new_rows.append(new_row)
    else:
        # If no matching row in combined, include augmented data as is
        new_rows.extend(date_entries.to_dict('records'))

# Create a DataFrame from the list of new rows
overall_data = pd.DataFrame(new_rows)

# Optional: Cleanup and reorder columns as needed

# Save the overall data to a new Excel file
overall_data.to_excel('Combined_overall_data.xlsx', index=False)

print("Overall data saved to overall_data.xlsx")

### Change the weather stations data with observation date similar to bird prediction

In [ ]:
import pandas as pd

# Load the data from the Excel file
df = pd.read_excel('Weather_Stations_combined_data.xlsx')

# Ensure 'Month' and 'Year' are strings to prevent pandas from interpreting them as numeric
df['Month'] = df['Month'].astype(str)
df['Year'] = df['Year'].astype(str)

# Convert 'Date' column to datetime, appending the 'Year', and handling errors
try:
    # Assuming the 'Date' column format is like "1 Nov", which doesn't include the day
    # Manually specify the format to ensure correct parsing
    df['Observation_Date'] = pd.to_datetime(df['Date'] + ' ' + df['Year'], format='%d %b %Y', errors='coerce')
except Exception as e:
    print(f"Error converting dates: {e}")

# Check for any conversion errors that resulted in NaT
if df['Observation_Date'].isnull().any():
    print("There are dates that couldn't be converted. They have been set to NaT.")

# Extract 'Year', 'Month', and 'Day' from 'Observation_Date'
df['Year'] = df['Observation_Date'].dt.year
df['Month'] = df['Observation_Date'].dt.month  # This will now be a numeric month
df['Day'] = df['Observation_Date'].dt.day

# Assuming 'Location' needs to be mapped or derived; for now, copying 'Station' to 'Location'
df['Location'] = df['Station']  # Adjust this as per your requirement to map or derive 'Location'

# Additional columns as specified
df['name'] = 'singapore'  # Assuming 'singapore' is a constant value for 'name'
df['datetime'] = df['Observation_Date']

# Retain all original columns in the DataFrame, now including the new or modified ones
# Ensure that all desired modifications are integrated without dropping any original data
columns_order = ['Station', 'Location', 'Observation_Date', 'Year', 'Month', 'Day', 'name', 'datetime'] + \
                [col for col in df.columns if col not in ['Station', 'Location', 'Observation_Date', 'Year', 'Month', 'Day', 'name', 'datetime']]
df_final = df[columns_order]

# Save the updated DataFrame to a new Excel file
df_final.to_excel('formatted_Weather_Stations_combined_data.xlsx', index=False)

print('Data saved to formatted_Weather_Stations_combined_data.xlsx')

### Combine the Final excel sheet data (both prediction and weather sightings)

In [ ]:

import pandas as pd

# Load data from Excel files
bird_df = pd.read_excel('Bird_Prediction_1.xlsx')
location_map_df = pd.read_excel('location_map_3.xlsx')
weather_df = pd.read_excel('Weather_Stations_combined_data_2.xlsx')

# Explicitly convert 'datetime' in bird_df and 'Observation_Date' in weather_df to datetime format
bird_df['datetime'] = pd.to_datetime(bird_df['datetime'], errors='coerce')
weather_df['Observation_Date'] = pd.to_datetime(weather_df['Observation_Date'], errors='coerce')

# Now, convert datetime to just date
bird_df['Observation_Date'] = bird_df['datetime'].dt.date
weather_df['Observation_Date'] = weather_df['Observation_Date'].dt.date

# Map station names from location_map to bird_df for consistent station naming
bird_df['Mapped_Station'] = bird_df['Location'].map(location_map_df.set_index('Bird_Location')['Bird_Station'].to_dict())

# Trim whitespace from station names to ensure consistent matching
weather_df['Station'] = weather_df['Station'].str.strip()
bird_df['Mapped_Station'] = bird_df['Mapped_Station'].str.strip()

# Merge bird_df with weather_df based on mapped station name and observation date
merged_df = pd.merge(
    bird_df, 
    weather_df, 
    left_on=['Mapped_Station', 'Observation_Date'], 
    right_on=['Station', 'Observation_Date'],
    how='left',
    suffixes=('', '_weather')
)

# Save the merged DataFrame to an Excel file
merged_df.to_excel('Final_Merged_Data.xlsx', index=False)

print("The data has been merged and saved successfully.")